# Convert taxa list to a pretty table

In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(tidyverse)
require(data.table)
require(foreach)
require(reactable)
require(htmltools)
require(htmlwidgets)
require(IRdisplay)
require(repr)
require(webshot2)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required packag

### Load  data and add metadata

In [2]:
# After parsing
path_prev_max_filt <- fread(str_glue("results/decontamination/curated_n122_global_decontamination_stats.parsed.csv"))

In [3]:
cov_df_parsed <- fread("results/irep_analysis/coverage_irep_results.parsed.csv") %>%
    select(prefix, taxa)
cov_df <- fread("results/irep_analysis/coverage_irep_results.raw.csv") %>%
    left_join(cov_df_parsed) %>%
    filter(bPTR > 0)

to_display <- path_prev_max_filt %>%
    left_join(cov_df, "taxa") %>%
    filter(bPTR > 0, org_group == "Bacteria") %>%
    mutate(site = ifelse(site == "oral", "mouth", site))

# to_display
to_display_morsels <- foreach(taxon = unique(to_display$taxa)) %do% {
        temp <- to_display %>% filter(taxa == taxon) 
        sites <- temp$site[!(temp$site %in% c("eye", "blood"))]
        sites <- sites[order(sites)]
#         sites <- paste0(sites, collapse = "<br>")
        sites <- paste0(sites, collapse = ", ")
        temp %>% 
            distinct(taxa, .keep_all = T) %>%
            mutate(site = sites)
}

final <- bind_rows(to_display_morsels)
final

Joining, by = "prefix"


taxa,genus.x,max_count,overall_prevalence,n_samples,max_bin,org_group,site,microbiol_class,prefix,⋯,file_name,genus.y,species,suffix1,suffix2,ORI,TER,bPTR,microbe_length,microbe_count
<chr>,<chr>,<int>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
Fusobacterium nucleatum,Fusobacterium,194199,0.001132759,10,>=500,Bacteria,"genitourinary, gut, mouth",Obligate anaerobe,Fusobacterium_nucleatum_NZ_LN831027.1,⋯,Fusobacterium_nucleatum_NZ_LN831027.1.fasta,Fusobacterium,nucleatum,NZ,LN831027.1.fasta,"201,937","1,345,205",1.675886,2443126,3657.7463
Neisseria subflava,Neisseria,15385,0.001585863,14,>=500,Bacteria,"gut, mouth",Aerobic,Neisseria_subflava_NZ_CP039887.1,⋯,Neisseria_subflava_NZ_CP039887.1.fasta,Neisseria,subflava,NZ,CP039887.1.fasta,"1,883,368","797,590",1.506405,2195659,1882.0952
Haemophilus parainfluenzae,Haemophilus,12183,0.002038967,18,>=500,Bacteria,"gut, mouth, respiratory tract",Facultative anaerobe,Haemophilus_parainfluenzae_NZ_GL872339.1,⋯,Haemophilus_parainfluenzae_NZ_GL872339.1.fasta,Haemophilus,parainfluenzae,NZ,GL872339.1.fasta,"492,915","1,443,748",1.172580,2112961,744.5671
Fannyhessea vaginae,Fannyhessea,10395,0.002378795,21,>=500,Bacteria,genitourinary,Obligate anaerobe,Fannyhessea_vaginae_NZ_UFSV01000001.1,⋯,Fannyhessea_vaginae_NZ_UFSV01000001.1.fasta,Fannyhessea,vaginae,NZ,UFSV01000001.1.fasta,"613,931","1,264,620",1.876599,1445974,707.5888
Staphylococcus epidermidis,Staphylococcus,9140,0.008608971,76,>=500,Bacteria,"gut, mouth, respiratory tract, skin",Facultative anaerobe,Staphylococcus_epidermidis_NZ_CP035288.1,⋯,Staphylococcus_epidermidis_NZ_CP035288.1.fasta,Staphylococcus,epidermidis,NZ,CP035288.1.fasta,"237,449","1,347,375",1.566844,2466502,734.9293
Lactobacillus crispatus,Lactobacillus,7799,0.010647938,94,>=500,Bacteria,"genitourinary, gut, mouth",Aerotolerant anaerobe,Lactobacillus_crispatus_NZ_CP039266.1,⋯,Lactobacillus_crispatus_NZ_CP039266.1.fasta,Lactobacillus,crispatus,NZ,CP039266.1.fasta,"328,627","1,239,005",1.571468,2023061,541.4163
Acinetobacter baumannii,Acinetobacter,7673,0.003171726,28,>=500,Bacteria,mouth,Aerobic,Acinetobacter_baumannii_NZ_CP043953.1,⋯,Acinetobacter_baumannii_NZ_CP043953.1.fasta,Acinetobacter,baumannii,NZ,CP043953.1.fasta,"3,869,401","1,967,227",1.896385,3972439,849.0606
Neisseria flavescens,Neisseria,3787,0.000566380,5,>=500,Bacteria,mouth,Aerobic,Neisseria_flavescens_NZ_CP039886.1,⋯,Neisseria_flavescens_NZ_CP039886.1.fasta,Neisseria,flavescens,NZ,CP039886.1.fasta,"1,737,155","716,802",1.380708,2231882,1721.0496
Rickettsia sp. Tillamook 23,Rickettsia,2923,0.000226552,2,>=500,Bacteria,,Obligate intracellular,Rickettsia_Tillamook_23_NZ_CP060138.1,⋯,Rickettsia_Tillamook_23_NZ_CP060138.1.fasta,Rickettsia,Tillamook,23,NZ,"240,832","888,370",1.346660,1438973,1197.8891


In [4]:
fread("results/irep_analysis/coverage_irep_results.raw.csv") %>%
    filter(!grepl("Achromo|Alcali|Pseu|Staphylococcus_haemolyticus|cohnii|Asper", prefix)) %>%
    summarise(range(perc_covered1))
#     distinct(prefix)

range(perc_covered1)
<dbl>
0.04072575
98.75583204


## Render table

In [5]:
# render functions
orange_pal <- function(x) rgb(colorRamp(c("khaki4", "khaki1"))(x), maxColorValue = 255)
green_pal <- function(x) scales::colour_ramp(c("white", "darkolivegreen1"), na.color = "white")(x)


In [9]:
parsed <- final %>%
    mutate(overall_prevalence = round(overall_prevalence * 100, 2),
           perc_covered1 = round(perc_covered1, 1),
           bPTR = round(bPTR, 2),
           emia = ifelse(taxa == "Rickettsia sp. Tillamook 23", F, T)) %>%
    select(npm_research_id, taxa, site, emia, pairs_assigned, overall_prevalence, bPTR) %>%
    arrange(desc(pairs_assigned))

rtable <- parsed %>%
  reactable(compact = T,
            borderless = F,
            pagination = F,
            style = list(fontFamily = "Arial", fontSize = "16px"),
            width = 1300,
            height = 450,
            columns = list(taxa = colDef(name = "Species",
                                         width = 250,
                                         align = "left",
                                         style = list(fontStyle = "italic")),
                           site = colDef(name = "Possible origin",
                                         width = 200,
                                         align = "center",
                                         html = T),
                           npm_research_id = colDef(name = "ID",
                                                    width = 100,
                                                    align = "center"),
                            
                           overall_prevalence = colDef(name = "Overall prevalence (%)",
                                                       align = "center",
                                                       width = 200,
                                                       style = function(value) {
                                                          normalized <- (value - min(parsed$overall_prevalence)) / (max(parsed$overall_prevalence) - min(parsed$overall_prevalence))
                                                          color <- green_pal(normalized)
                                                          list(background = color)
                                                       }),
                           pairs_assigned = colDef(name = "No. of read pairs assigned",
                                                       align = "center"),
                           bPTR = colDef(name = "PTR",
                                         width = 80,
                                             align = "center",
                                             style = function(value) {
                                                 normalised <- (value - min(parsed$bPTR, na.rm = T)) / 
                                                   (max(parsed$bPTR, na.rm = T) - min(parsed$bPTR, na.rm = T))
                                                 color <- green_pal(normalised)
                                                 list(background = color)
                                             }),                        
                           emia = colDef(name = "Reported in blood",
                                         align = "center",
                                         width = 150,
#                                          style = list(borderRight = "1px solid rgba(0, 0, 0, 0.5)"),
                                         cell = function(value) {
                                             # Render as an X mark or check mark
                                             if (value) "\u2714\ufe0f" else "\u274c"
                                         })
            )
  )

html <- "results/irep_analysis/parsed_irep_results.html"
saveWidget(rtable, html)